In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

c:\Users\Sanjana Rayarala\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
#Encoder
class Encoder(nn.Module):
    def __init__(self,input_size, hidden_size, num_layers):
        super(Encoder, self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.num_layers= num_layers
        self.lstm= nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    
    def forward(self, inputs):
        outputs, (h,c)= self.lstm(inputs)
        return h,c
    

In [3]:
#Decoder
class Decoder(nn.Module):
    def __init__(self, output_size, hidden_size, num_layers):
        super(Decoder, self).__init__()
        self.output_size = output_size
        self.hidden_size= hidden_size
        self.num_layers= num_layers
        self.lstm= nn.LSTM(output_size, hidden_size, num_layers, batch_first=True)
        self.fc= nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        outputs, (hidden,cell)= self.lstm(x, (hidden, cell))
        prediction = self.fc(outputs.squeeze(1))
        return prediction, hidden, cell

In [11]:
class Seq2Seq(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, num_layers)
        self.decoder = Decoder(output_size, hidden_size, num_layers)

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: (batch_size, src_len, input_dim)
        # trg: (batch_size, trg_len, output_dim)

        batch_size, trg_len, _ =trg.shape
        outputs= torch.zeros(batch_size, trg_len, trg.shape[-1]).to(src.device)

        hidden, cell = self.encoder(src)

        input = trg[:,0,:].unsqueeze(1)

        for t in range(1, trg_len):
            output, hidden, cell =self.decoder(input, hidden, cell)
            outputs[:,t,:] = output

            if torch.rand(1).item()< teacher_forcing_ratio:
                input= trg[:,t,:].unsqueeze(1)
            else:
                input = output.unsqueeze(1)
        return outputs

In [9]:
def train(model, train_loader, epochs=10, lr=1e-3):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()  # For regression tasks (can be changed for classification tasks)
    
    for epoch in range(epochs):
        total_loss = 0
        for batch_idx, (src, trg) in enumerate(train_loader):
            optimizer.zero_grad()
            # Forward pass
            output = model(src, trg)
            # Compute loss
            loss = criterion(output, trg)
            total_loss += loss.item()
            # Backpropagation
            loss.backward()
            optimizer.step()
        
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

In [6]:
from torch.utils.data import DataLoader, TensorDataset

# Example Data (Random for demonstration)
src_data = torch.rand(100, 10, 5)  # (100 samples, 10 time steps, 5 features)
trg_data = torch.rand(100, 10, 5)  # (100 samples, 10 time steps, 5 target features)

# Create a TensorDataset and DataLoader
dataset = TensorDataset(src_data, trg_data)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)


In [ ]:
#Initialize Model
input_dim = 5
output_dim = 5
hidden_dim = 32
num_layers = 2

model = Seq2Seq(input_dim, output_dim, hidden_dim, num_layers)

# Train the Model
train(model, train_loader, epochs=200, lr=1e-3)


Epoch [1/200], Loss: 0.2669
Epoch [2/200], Loss: 0.2201
Epoch [3/200], Loss: 0.1606
Epoch [4/200], Loss: 0.1203
Epoch [5/200], Loss: 0.1125
Epoch [6/200], Loss: 0.1090
Epoch [7/200], Loss: 0.1095
Epoch [8/200], Loss: 0.1069
Epoch [9/200], Loss: 0.1057
Epoch [10/200], Loss: 0.1066
Epoch [11/200], Loss: 0.1067
Epoch [12/200], Loss: 0.1074
Epoch [13/200], Loss: 0.1058
Epoch [14/200], Loss: 0.1068
Epoch [15/200], Loss: 0.1059
Epoch [16/200], Loss: 0.1065
Epoch [17/200], Loss: 0.1076
Epoch [18/200], Loss: 0.1066
Epoch [19/200], Loss: 0.1090
Epoch [20/200], Loss: 0.1071
Epoch [21/200], Loss: 0.1060
Epoch [22/200], Loss: 0.1068
Epoch [23/200], Loss: 0.1060
Epoch [24/200], Loss: 0.1058
Epoch [25/200], Loss: 0.1071
Epoch [26/200], Loss: 0.1073
Epoch [27/200], Loss: 0.1066
Epoch [28/200], Loss: 0.1078
Epoch [29/200], Loss: 0.1058
Epoch [30/200], Loss: 0.1060
Epoch [31/200], Loss: 0.1055
Epoch [32/200], Loss: 0.1067
Epoch [33/200], Loss: 0.1062
Epoch [34/200], Loss: 0.1066
Epoch [35/200], Loss: 0